#  图卷积介绍

在本教程中，我们将学习更多关于"图卷积"的知识。这些是处理分子数据最强大的深度学习工具之一，因为分子可以自然地被看作是图。

![Molecular Graph](https://github.com/deepchem/deepchem/blob/master/examples/tutorials/assets/basic_graphs.gif?raw=1)

请注意我们在高中时习惯的那种标准化学图是如何自然地将分子可视化为图形的。在本教程的剩余部分，我们将更详细地进行研究，这将使我们更深入地了解这些系统的工作原理。

## Colab

本教程和目录中的其余部分都是在 Google colab 中完成。如果您想在 colab 中打开此笔记本，您可以点击以下链接。

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/BioGavin/deepchem/blob/master/examples/tutorials-zh/Introduction_to_Graph_Convolutions.ipynb)



In [ ]:
!pip install --pre deepchem

# 什么是图卷积？

考虑一个标准的卷积神经网络（CNN），即通常用于处理图像的那种。输入是一个像素的网格，每个像素都有一个数据值的向量，例如红、绿、蓝三色通道。数据通过一系列的卷积层，每一层都将来自一个像素和它的邻居的数据结合起来，为这些像素产生一个新的数据向量。前期层检测小规模的局部模式，而后期层检测更大、更抽象的模式。卷积层通常与池化层交替进行，后者对局部区域进行一些操作，如最大池化或最小池化。

图卷积也是类似的，但它们是在图上操作的。它们从图的每个节点的数据向量开始（例如，代表原子化学属性的节点数据）。卷积层和池化层汇聚来自相连节点的信息（例如，相互结合的原子），为每个节点产生一个新的数据向量。

# 训练一个 GraphConvModel

让我们使用 MoleculeNet 套件来加载 Tox21 数据集。为了使用图卷积网络，我们将特征生成器选项设置为 "GraphConv"。MoleculeNet 调用返回一个训练集、一个验证集和一个测试集供我们使用。它还返回`tasks`，一个任务名称的列表，以及 `transformers`，一个应用于预处理数据集的数据转换器的列表。(大多数深度网络是相当棘手的，需要一组数据转换器来确保训练的稳定进行。)

In [1]:
import deepchem as dc

tasks, datasets, transformers = dc.molnet.load_tox21(featurizer='GraphConv')
train_dataset, valid_dataset, test_dataset = datasets

现在让我们在这个数据集上训练一个图卷积网络。DeepChem 有一个 `GraphConvModel` 类，为了方便用户，它将一个标准的图卷积架构进行了包装。让我们实例化这个类取得一个对象并在我们的数据集上训练它。

In [2]:
n_tasks = len(tasks)
model = dc.models.GraphConvModel(n_tasks, mode='classification')
model.fit(train_dataset, nb_epoch=50)

0.28185401916503905

让我们试着评估一下我们所训练的模型的性能。为此，我们需要定义一个指标，一个衡量模型性能的标准。`dc.metrics` 已经包含了一个指标集合。对于这个数据集，标准的做法是使用 ROC-AUC 分数，即接收者操作特征曲线下的面积（衡量精度和召回率之间的权衡）。幸运的是，ROC-AUC 分数已经在 DeepChem 中可用。

为了衡量模型在这个指标下的性能，我们可以很方便地使用 `model.evaluate()` 函数。

In [3]:
metric = dc.metrics.Metric(dc.metrics.roc_auc_score)
print('Training set score:', model.evaluate(train_dataset, [metric], transformers))
print('Test set score:', model.evaluate(test_dataset, [metric], transformers))

Training set score: {'roc_auc_score': 0.96959686893055}
Test set score: {'roc_auc_score': 0.795793783300876}


结果还不错，`GraphConvModel` 非常好用。但是引擎盖下发生了什么？我们可以自己构建 GraphConvModel 吗？ 当然！ DeepChem 为图卷积中涉及的所有计算提供 Keras 层。 我们将应用 DeepChem 的以下层。

- `GraphConv` 层：该层实现图卷积。图卷积以非线性方式将每个节点的特征向量与相邻节点的特征向量组合在一起。这将信息“混合”在图的本地邻域中。

- `GraphPool` 层：该层对邻域中原子的特征向量进行最大池化。您可以将此层视为类似于 2D 卷积的最大池化层，但它在图上运行。

- `GraphGather` 层：许多图卷积网络操作每个图节点的特征向量。例如，对于一个分子，每个节点可能代表一个原子，网络将操纵原子特征向量，总结原子的局部化学性质。 但是，在应用程序结束时，我们可能希望使用分子级别的特征表示，该层通过组合所有节点级特征向量来创建图级特征向量。

除此之外，我们将应用标准的神经网络层，例如 [Dense](https://keras.io/api/layers/core_layers/dense/), [BatchNormalization](https://keras.io/api/layers/normalization_layers/batch_normalization/) 和 [Softmax](https://keras.io/api/layers/activation_layers/softmax/) 层。

In [4]:
from deepchem.models.layers import GraphConv, GraphPool, GraphGather
import tensorflow as tf
import tensorflow.keras.layers as layers

batch_size = 100

class MyGraphConvModel(tf.keras.Model):

  def __init__(self):
    super(MyGraphConvModel, self).__init__()
    self.gc1 = GraphConv(128, activation_fn=tf.nn.tanh)
    self.batch_norm1 = layers.BatchNormalization()
    self.gp1 = GraphPool()

    self.gc2 = GraphConv(128, activation_fn=tf.nn.tanh)
    self.batch_norm2 = layers.BatchNormalization()
    self.gp2 = GraphPool()

    self.dense1 = layers.Dense(256, activation=tf.nn.tanh)
    self.batch_norm3 = layers.BatchNormalization()
    self.readout = GraphGather(batch_size=batch_size, activation_fn=tf.nn.tanh)

    self.dense2 = layers.Dense(n_tasks*2)
    self.logits = layers.Reshape((n_tasks, 2))
    self.softmax = layers.Softmax()

  def call(self, inputs):
    gc1_output = self.gc1(inputs)
    batch_norm1_output = self.batch_norm1(gc1_output)
    gp1_output = self.gp1([batch_norm1_output] + inputs[1:])

    gc2_output = self.gc2([gp1_output] + inputs[1:])
    batch_norm2_output = self.batch_norm1(gc2_output)
    gp2_output = self.gp2([batch_norm2_output] + inputs[1:])

    dense1_output = self.dense1(gp2_output)
    batch_norm3_output = self.batch_norm3(dense1_output)
    readout_output = self.readout([batch_norm3_output] + inputs[1:])

    logits_output = self.logits(self.dense2(readout_output))
    return self.softmax(logits_output)

我们现在可以更清楚地看到正在发生什么。有两个卷积块，每个卷积块包括一个 "GraphConv"，然后是批量归一化，接着是一个 "GraphPool" 来做最大池化。我们最后有一个稠密层，另一个批量归一化，一个 `GraphGather` 来合并所有不同节点的数据，最后一个密集层来产生全局输出。

现在让我们创建 DeepChem 模型，它将是我们刚刚创建的 Keras 模型的一个封装器。我们还将指定损失函数，以便模型知道要最小化的目标。

In [5]:
model = dc.models.KerasModel(MyGraphConvModel(), loss=dc.models.losses.CategoricalCrossEntropy())

这个模型的输入是什么？图卷积需要对每个分子进行完整的描述，包括节点（原子）的列表和对哪些节点之间相互结合的描述。事实上，如果我们检查数据集，就会发现特征数组包含了 `ConvMol` 类型的 Python 对象。

In [6]:
test_dataset.X[0]

模型期望数组作为它们的输入，而不是 Python 对象。我们必须将 `ConvMol` 对象转换成 `GraphConv`、`GraphPool` 和 `GraphGather` 层所期望的特定数组。幸运的是，`ConvMol` 类包括了这样做的代码，以及将所有的分子合并到一个批次中来创建一个单一的数组。

以下代码创建了一个 Python 生成器，给定一批数据，生成输入、标签和权重的列表，其值为 Numpy 数组。`atom_features` 为每个原子保存一个长度为 75 的特征向量。其他的输入是为了支持 TensorFlow 的 minibatching。`degree_slice` 是一个索引便利，使其能够轻松地从所有具有指定程度的分子中找到原子。`membership` 决定了原子在分子中的成员资格（原子 `i` 属于分子 `membership[i]`）。`deg_adjs` 是一个包含按原子程度分组的邻接矩阵的列表。更多详细的内容，请查看[代码](https://github.com/deepchem/deepchem/blob/master/deepchem/feat/mol_graphs.py)。

In [7]:
from deepchem.metrics import to_one_hot
from deepchem.feat.mol_graphs import ConvMol
import numpy as np

def data_generator(dataset, epochs=1):
  for ind, (X_b, y_b, w_b, ids_b) in enumerate(dataset.iterbatches(batch_size, epochs,
                                                                   deterministic=False, pad_batches=True)):
    multiConvMol = ConvMol.agglomerate_mols(X_b)
    inputs = [multiConvMol.get_atom_features(), multiConvMol.deg_slice, np.array(multiConvMol.membership)]
    for i in range(1, len(multiConvMol.get_deg_adjacency_lists())):
      inputs.append(multiConvMol.get_deg_adjacency_lists()[i])
    labels = [to_one_hot(y_b.flatten(), 2).reshape(-1, n_tasks, 2)]
    weights = [w_b]
    yield (inputs, labels, weights)

现在，我们可以使用 `fit_generator(generator)` 来训练模型，它将使用我们定义的数据生成器来训练模型。

In [1]:
model.fit_generator(data_generator(train_dataset, epochs=50))

NameError: name 'model' is not defined

现在我们已经训练了我们的图卷积方法，让我们来评估其性能。我们又要使用我们定义的数据生成器来评估模型的性能。

In [9]:
print('Training set score:', model.evaluate_generator(data_generator(train_dataset), [metric], transformers))
print('Test set score:', model.evaluate_generator(data_generator(test_dataset), [metric], transformers))

Training set score: {'roc_auc_score': 0.8425638289185731}
Test set score: {'roc_auc_score': 0.7378436684114341}


成功了! 我们构建的模型与 `GraphConvModel` 的行为几乎相同。如果你想建立你自己的自定义模型，你可以按照我们在这里提供的例子来做。我们希望很快就能看到来自你的令人兴奋的构建!

# 恭喜！是时候加入社区了！

恭喜您完成本教程笔记本！如果您喜欢本教程并希望继续使用 DeepChem，我们鼓励您完成本系列的其余教程。您还可以通过以下方式帮助 DeepChem 社区：

## 在 [GitHub](https://github.com/deepchem/deepchem) 上为 DeepChem 点亮小星星
这有助于大家建立对 DeepChem 项目和我们正在尝试构建的开源药物发现工具的共识。

## 加入 DeepChem Gitter
DeepChem [Gitter](https://gitter.im/deepchem/Lobby) 聚集了许多对生命科学深度学习感兴趣的科学家、开发人员和爱好者，欢迎加入！